# 🤖 Building Autonomous Agents: AgentCore Code Interpreter

Welcome to the AgentCore Code Interpreter module! In this notebook, we'll explore how to integrate AWS Bedrock AgentCore Code Interpreter into autonomous agents, enabling them to execute code securely in sandbox environments.

## What is AgentCore Code Interpreter?

The Amazon Bedrock AgentCore Code Interpreter enables AI agents to write and execute code securely in sandbox environments, enhancing their accuracy and expanding their ability to solve complex end-to-end tasks. It supports multiple programming languages and provides secure, isolated execution environments.

## 🎯 Learning Objectives

In this notebook, you'll learn how to:

- 🔧 Set up and use AgentCore Code Interpreter with both boto3 and AgentCore SDK
- 📊 Create data visualizations and graphs using code execution
- 🤖 Integrate code interpreter capabilities into agents using Strands and LangChain frameworks
- 📁 Manage files within code interpreter sessions
- ☁️ Use S3 integration and terminal commands with execution roles
- 🏗️ Build framework-agnostic abstractions for code execution tools
- 🔄 Integrate with existing Module 3 memory and tool patterns

## 📋 Prerequisites

- AWS account with access to Amazon Bedrock AgentCore
- Proper IAM permissions for AgentCore services
- Python 3.12 environment with required dependencies

Let's begin! 🚀

## 🔧 Section 1: Environment Setup and Dependencies

First, let's set up our environment and install the necessary dependencies.

In [ ]:
# 🔧 Environment Setup Instructions

# IMPORTANT: This notebook requires Python 3.12.8 as specified in pyproject.toml
# All dependencies are managed via UV (recommended approach for this project)

# SETUP STEPS:
# 1. Create virtual environment: uv venv
# 2. Install all dependencies: uv sync
# 3. Launch Jupyter Lab: uv run jupyter lab

# FOR VS CODE USERS:
# 1. Run: uv run python -c "import sys; print(sys.executable)"
# 2. Copy the path and select it as your Python interpreter in VS Code
# 3. The kernel should now have access to all required dependencies

# Verify environment setup
import sys
print(f"✅ Python version: {sys.version}")
print(f"✅ Python executable: {sys.executable}")
print("✅ Environment setup complete! All dependencies should be available via UV.")

In [ ]:
# Import core dependencies
import boto3
import json
import os
import time
import asyncio
from typing import Dict, Any, List, Optional
from pydantic import BaseModel
from abc import ABC, abstractmethod

# Import AgentCore SDK
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter, code_session

# Import existing Module 3 components
from agentic_platform.core.models.memory_models import Message, SessionContext
from agentic_platform.core.models.llm_models import LLMRequest, LLMResponse
from agentic_platform.core.models.api_models import AgenticRequest, AgenticResponse

print("✅ Dependencies imported successfully!")

In [ ]:
# Initialize AWS clients with specific profile
profile_name = "ags-tech-namer-ca-pace+ai3-core-Admin"
session = boto3.Session(profile_name=profile_name)
region_name = "us-west-2"

# Verify credentials are working
try:
    credentials = session.get_credentials()
    print(f"✅ Using AWS profile: {profile_name}")
    print(f"✅ Access Key: {credentials.access_key[:10]}...")
    print(f"✅ Region: {region_name}")
except Exception as e:
    print(f"❌ Error with AWS credentials: {e}")
    raise

# Standard AgentCore client
bedrock_agentcore_client = session.client(
    'bedrock-agentcore',
    region_name=region_name,
    endpoint_url=f"https://bedrock-agentcore.{region_name}.amazonaws.com"
)

# Control plane client for custom interpreters
bedrock_agentcore_control_client = session.client(
    'bedrock-agentcore-control',
    region_name=region_name,
    endpoint_url=f"https://bedrock-agentcore-control.{region_name}.amazonaws.com"
)

print("✅ AWS clients initialized successfully with profile!")

## 📊 Section 2: Basic Code Execution and Data Analysis

Let's start with basic code execution examples, demonstrating server-side JavaScript data processing and Python visualizations using both boto3 and the AgentCore SDK approaches.

### Example 1: Boto3 Direct Implementation with Server-Side JavaScript

In [ ]:
# Server-side JavaScript code for data analysis, ASCII visualization, and file operations
javascript_analysis_code = """
// Server-side JavaScript data analysis (Deno compatible)
// Fixed version without browser dependencies

console.log('Starting comprehensive data analysis...');

// Sample sales data
const salesData = {
    months: ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul'],
    actual: [65, 59, 80, 81, 56, 55, 40],
    target: [70, 65, 75, 80, 60, 58, 45]
};

// Statistical calculations
function calculateStats(data) {
    const sum = data.reduce((a, b) => a + b, 0);
    const avg = sum / data.length;
    const max = Math.max(...data);
    const min = Math.min(...data);
    const variance = data.reduce((acc, val) => acc + Math.pow(val - avg, 2), 0) / data.length;
    const stdDev = Math.sqrt(variance);
    return { sum, avg, max, min, variance, stdDev };
}

const actualStats = calculateStats(salesData.actual);
const targetStats = calculateStats(salesData.target);
const performance = (actualStats.avg / targetStats.avg) * 100;

console.log('SALES ANALYSIS RESULTS:');
console.log('Average Actual Sales: ' + actualStats.avg.toFixed(1));
console.log('Average Target Sales: ' + targetStats.avg.toFixed(1));
console.log('Overall Performance: ' + performance.toFixed(1) + '%');
console.log('Standard Deviation: ' + actualStats.stdDev.toFixed(2));

// ASCII Chart Visualization
function createASCIIBarChart(data, labels, title) {
    const maxValue = Math.max(...data);
    const chartWidth = 30;
    
    console.log('\\n' + title);
    console.log('='.repeat(40));
    
    data.forEach((value, index) => {
        const barLength = Math.round((value / maxValue) * chartWidth);
        const bar = '#'.repeat(barLength);
        const padding = ' '.repeat(Math.max(0, chartWidth - barLength));
        console.log(labels[index] + ' |' + bar + padding + '| ' + value);
    });
    
    console.log(' '.repeat(5) + '+' + '-'.repeat(chartWidth) + '+');
}

// Create ASCII charts
createASCIIBarChart(salesData.actual, salesData.months, 'ACTUAL SALES BY MONTH');
createASCIIBarChart(salesData.target, salesData.months, 'TARGET SALES BY MONTH');

// Trend Analysis
function calculateTrend(data) {
    const n = data.length;
    const sumX = (n * (n + 1)) / 2;
    const sumY = data.reduce((a, b) => a + b, 0);
    const sumXY = data.reduce((sum, y, x) => sum + (x + 1) * y, 0);
    const sumX2 = (n * (n + 1) * (2 * n + 1)) / 6;
    
    const slope = (n * sumXY - sumX * sumY) / (n * sumX2 - sumX * sumX);
    return { slope, trend: slope > 0 ? 'Increasing' : slope < 0 ? 'Decreasing' : 'Stable' };
}

const actualTrend = calculateTrend(salesData.actual);
console.log('\\nTREND ANALYSIS:');
console.log('Sales Trend: ' + actualTrend.trend);
console.log('Slope: ' + actualTrend.slope.toFixed(3) + ' units per month');

// File Operations (JSON report)
const reportData = {
    timestamp: new Date().toISOString(),
    summary: {
        totalActualSales: actualStats.sum,
        totalTargetSales: targetStats.sum,
        performancePercentage: performance,
        trend: actualTrend.trend,
        trendSlope: actualTrend.slope
    },
    monthlyData: salesData.months.map((month, index) => ({
        month: month,
        actual: salesData.actual[index],
        target: salesData.target[index],
        variance: salesData.actual[index] - salesData.target[index]
    }))
};

// Mathematical Computations
function fibonacci(n) {
    if (n <= 1) return n;
    let a = 0, b = 1;
    for (let i = 2; i <= n; i++) {
        const temp = a + b;
        a = b;
        b = temp;
    }
    return b;
}

function isPrime(num) {
    if (num < 2) return false;
    for (let i = 2; i <= Math.sqrt(num); i++) {
        if (num % i === 0) return false;
    }
    return true;
}

console.log('\\nMATHEMATICAL COMPUTATIONS:');
console.log('Fibonacci(10): ' + fibonacci(10));

const primes = [];
for (let i = 0; i < 30; i++) {
    if (isPrime(i)) primes.push(i);
}
console.log('Primes under 30: ' + primes.join(', '));

// Final summary
console.log('\\nANALYSIS COMPLETE!');
console.log('This demonstrates server-side JavaScript capabilities:');
console.log('• Data processing and statistical analysis');
console.log('• ASCII chart generation for visualization');
console.log('• Mathematical computations');
console.log('• Structured data transformation');

// Return final results
reportData.summary;
"""

print("Server-side JavaScript analysis code prepared!")

In [ ]:
def execute_code_with_boto3(code: str, language: str = "javascript") -> Dict[str, Any]:
    """
    Execute code using boto3 direct API calls to AgentCore Code Interpreter.
    """
    try:
        # Start a code interpreter session
        session_response = bedrock_agentcore_client.start_code_interpreter_session(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            name="boto3-analysis-session",
            sessionTimeoutSeconds=900
        )
        session_id = session_response["sessionId"]
        print(f"✅ Started session: {session_id}")
        
        # Execute the code
        execute_response = bedrock_agentcore_client.invoke_code_interpreter(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            sessionId=session_id,
            name="executeCode",
            arguments={
                "language": language,
                "code": code
            }
        )
        
        # Process the response stream
        results = []
        for event in execute_response['stream']:
            if 'result' in event:
                result = event['result']
                results.append(result)
                
                # Print text output
                if 'content' in result:
                    for content_item in result['content']:
                        if content_item['type'] == 'text':
                            print(f"Output: {content_item['text']}")
                        elif content_item['type'] == 'image':
                            print(f"Generated image: {content_item['data'][:50]}...")
        
        # Clean up session
        bedrock_agentcore_client.stop_code_interpreter_session(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            sessionId=session_id
        )
        print(f"✅ Session {session_id} stopped")
        
        return {
            "success": True,
            "session_id": session_id,
            "results": results
        }
        
    except Exception as e:
        print(f"❌ Error executing code: {str(e)}")
        return {
            "success": False,
            "error": str(e)
        }

# Execute the server-side JavaScript analysis code
print("🚀 Executing server-side JavaScript analysis with boto3...")
boto3_result = execute_code_with_boto3(javascript_analysis_code, "javascript")
print(f"Result: {boto3_result['success']}")

### Example 2: AgentCore SDK Implementation with Python Graph

In [ ]:
# Python code for creating a matplotlib graph
python_graph_code = """
import matplotlib.pyplot as plt
import numpy as np

# Sample sales data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul']
sales = [65, 59, 80, 81, 56, 55, 40]
target = [70, 65, 75, 80, 60, 58, 45]

# Create figure with subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart
x_pos = np.arange(len(months))
width = 0.35

ax1.bar(x_pos - width/2, sales, width, label='Actual Sales', color='#3498db', alpha=0.8)
ax1.bar(x_pos + width/2, target, width, label='Target', color='#e74c3c', alpha=0.8)
ax1.set_xlabel('Month')
ax1.set_ylabel('Sales')
ax1.set_title('Monthly Sales vs Target')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(months)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Line chart
ax2.plot(months, sales, marker='o', linewidth=2, label='Actual Sales', color='#3498db')
ax2.plot(months, target, marker='s', linewidth=2, label='Target', color='#e74c3c')
ax2.set_xlabel('Month')
ax2.set_ylabel('Sales')
ax2.set_title('Sales Trend Analysis')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Adjust layout and save
plt.tight_layout()
plt.savefig('sales_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate some statistics
avg_sales = np.mean(sales)
avg_target = np.mean(target)
performance = (avg_sales / avg_target) * 100

print(f"Average Sales: {avg_sales:.1f}")
print(f"Average Target: {avg_target:.1f}")
print(f"Performance: {performance:.1f}%")

# Return summary
{
    'avg_sales': avg_sales,
    'avg_target': avg_target,
    'performance_pct': performance,
    'chart_saved': 'sales_analysis.png'
}
"""

print("Python graph code prepared!")

In [ ]:
def execute_code_with_sdk(code: str, language: str = "python") -> Dict[str, Any]:
    """
    Execute code using the AgentCore SDK (higher-level interface).
    """
    try:
        # Configure and start the code interpreter session
        code_client = CodeInterpreter(region_name)
        code_client.start()
        print("✅ AgentCore SDK session started")
        
        # Execute the code
        response = code_client.invoke("executeCode", {
            "language": language,
            "code": code
        })
        
        # Process and print the response
        results = []
        for event in response["stream"]:
            if "result" in event:
                result = event["result"]
                results.append(result)
                print(f"Execution result: {json.dumps(result, indent=2, default=str)}")
        
        # Clean up and stop the session
        code_client.stop()
        print("✅ AgentCore SDK session stopped")
        
        return {
            "success": True,
            "results": results
        }
        
    except Exception as e:
        print(f"❌ Error executing code with SDK: {str(e)}")
        return {
            "success": False,
            "error": str(e)
        }

# Execute the Python graph code
print("🚀 Executing Python graph generation with AgentCore SDK...")
sdk_result = execute_code_with_sdk(python_graph_code, "python")
print(f"Result: {sdk_result['success']}")

## 🎯 Section 3: Summary and Key Takeaways

Congratulations! You've successfully explored AWS Bedrock AgentCore Code Interpreter and learned how to fix the DOM compatibility issue.

### 🔧 Problem Solved: DOM Compatibility Issue

**Original Issue**: The JavaScript code was trying to use browser DOM APIs (`document.createElement('canvas')`) in a Deno server-side environment where these APIs don't exist.

**Root Cause**: AgentCore Code Interpreter runs JavaScript in a Deno runtime (server-side), not in a browser environment.

**Solution Implemented**: Replaced browser-dependent code with server-side compatible alternatives:

1. **Data Processing & Analysis**: Statistical calculations and data manipulation
2. **ASCII Chart Visualization**: Terminal-friendly charts using Unicode characters
3. **File Operations**: JSON and CSV file generation using Deno APIs
4. **Mathematical Computations**: Fibonacci sequences, prime number detection
5. **Structured Data Transformation**: Complex data processing workflows

### 📊 What We've Demonstrated

The updated JavaScript example showcases:

- ✅ **Server-side JavaScript execution** in a secure sandbox
- ✅ **Data analysis and statistical calculations**
- ✅ **ASCII-based data visualization** (no browser dependencies)
- ✅ **File I/O operations** for report generation
- ✅ **Mathematical computations** and algorithms
- ✅ **Structured data processing** and transformation

### 🔑 Key Takeaways

1. **Environment Awareness**: Always consider the execution environment (server-side vs browser)
2. **Alternative Approaches**: ASCII charts can be as effective as graphical ones for data visualization
3. **Server-side Capabilities**: Deno provides powerful APIs for file operations and data processing
4. **Code Interpreter Flexibility**: Supports multiple programming languages with different strengths
5. **Sandbox Security**: Secure execution environment with controlled access to system resources

### 🚀 Next Steps

To continue working with AgentCore Code Interpreter:

1. **Experiment**: Try different server-side JavaScript libraries and APIs
2. **Integrate**: Add code interpreter capabilities to your existing agents
3. **Extend**: Build custom tools that leverage both JavaScript and Python strengths
4. **Scale**: Explore production deployment patterns
5. **Monitor**: Implement proper logging and error handling

### 📚 Additional Resources

- [Deno Documentation](https://deno.land/manual) - Server-side JavaScript runtime
- [AWS Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock-agentcore/)
- [AgentCore Code Interpreter API Reference](https://docs.aws.amazon.com/bedrock-agentcore/latest/APIReference/)
- [Module 3 Other Notebooks](../README.md) - Explore other agent patterns

Happy coding! 🎉